In [34]:
from typing import TypedDict, List, Annotated
from langgraph.graph import StateGraph, START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import SystemMessage, HumanMessage, AIMessage
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import os

In [10]:
load_dotenv('api_secret.env')
api_key = os.environ.get('API_KEY')


In [ ]:
llm = ChatGoogleGenerativeAI(model='gemini-2.5-flash', google_api_key=api_key)

In [40]:
class figures(BaseModel):
    amount : float = Field(description='What is the amount the user is looking for?')


In [ ]:
structured_llm = llm.with_structured_output(figures)

In [32]:
class Loan_agent_state(TypedDict):
    user_message : str
    agent_message : str
    interest : float
    amount : float
    tenure : int
    name: str
    phone: str
    address : str
    credit_score : int
    pre_approved_amount : int

In [ ]:
def Master(state: Loan_agent_state):
    directive = SystemMessage(content="You are a friendly and conversational banking assistant whose goal is to understand a customer's intentions behind exploring a personal loan — without discussing numbers or eligibility. " \
    "Greet the user warmly, ask simple and natural questions to learn what they need the loan for (like travel, education, emergencies, or lifestyle goals), and respond with empathy and encouragement. " \
    "Keep the tone approachable, positive, and human — like a helpful friend guiding them through their options. " \
    "Focus only on understanding their purpose and building trust, not selling or quoting figures."\
    "However if the user already wants to get a loan and already has his own figures like amount, rate and interest etc, then negotiate the numbers with him based on the following context : You are a highly skilled and persuasive financial advisor working for a leading bank. " \
    "Your mission is to understand each customer's unique financial situation, identify their core needs, and confidently guide them toward taking a personal loan that best aligns with their goals. " \
    "Communicate with authority, empathy, and clarity — ask insightful questions to uncover motivations such as education expenses, medical emergencies, debt consolidation, or lifestyle improvements. " \
    "Once you understand their priorities, negotiate terms strategically, address concerns with transparency, and demonstrate how the personal loan can genuinely improve their financial well-being through flexibility, better cash flow, or responsible credit building. " \
    "Maintain a consultative yet assertive tone — professional, approachable, and persuasive — ensuring the customer feels understood and empowered, not pressured. " \
    "Use emotional intelligence and data-driven reasoning to build trust, overcome objections, and clearly present the loan as a smart, beneficial decision. " \
    "Conclude every conversation with confidence, guiding the customer toward an actionable next step such as completing the loan application, ensuring they view the offer as a pathway to financial empowerment and peace of mind.")

    response = llm.invoke([directive, HumanMessage(content=state['user_message'])]).content
    state['agent_message'] = response

    return state

In [31]:
def Sales(state: Loan_agent_state) -> Loan_agent_state: 
    
    directive = SystemMessage(content="You are a highly skilled and persuasive financial advisor working for a leading bank. " \
    "Your mission is to understand each customer's unique financial situation, identify their core needs, and confidently guide them toward taking a personal loan that best aligns with their goals. " \
    "Communicate with authority, empathy, and clarity — ask insightful questions to uncover motivations such as education expenses, medical emergencies, debt consolidation, or lifestyle improvements. " \
    "Once you understand their priorities, negotiate terms strategically, address concerns with transparency, and demonstrate how the personal loan can genuinely improve their financial well-being through flexibility, better cash flow, or responsible credit building. " \
    "Maintain a consultative yet assertive tone — professional, approachable, and persuasive — ensuring the customer feels understood and empowered, not pressured. " \
    "Use emotional intelligence and data-driven reasoning to build trust, overcome objections, and clearly present the loan as a smart, beneficial decision. " \
    "Conclude every conversation with confidence, guiding the customer toward an actionable next step such as completing the loan application, ensuring they view the offer as a pathway to financial empowerment and peace of mind.")

    response = llm.invoke([directive, HumanMessage(content=state['user_message'])]).content

    state['agent_message'] = response

    return state
    



In [ ]:
def Verification(state: Loan_agent_state):
    return 0


    

In [25]:
def UnderWriting(state: Loan_agent_state):
    return 0

In [26]:
def Sanction_letter_generate(state: Loan_agent_state):
    return 0

In [ ]:
def Routing(state: Loan_agent_state):
    if state['user_message'] == 'exit':
        return 'end'
    else:
        return 0
        

In [29]:
graph = StateGraph(Loan_agent_state)

#adding nodes
graph.add_node('Master', Master)
graph.add_node('Sales', Sales)
graph.add_node('Verification', Verification)
graph.add_node('UnderWriting', UnderWriting)
graph.add_node('Sanction_letter_generate', Sanction_letter_generate)

#adding edges
graph.add_edge(START, 'Master')
graph.add_edge('Master','Sales')
graph.add_edge('Sales','Verification')
graph.add_edge('Verification','UnderWriting')
graph.add_edge('UnderWriting','Sanction_letter_generate')
graph.add_edge('Sanction_letter_generate',END)

#compiling the graph

app = graph.compile()



